In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
import requests
import json
from pathlib import Path
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
#from sklearn.preprocessing import StandardScaler
#from sklearn.ensemble import GradientBoostingClassifier
from collections import Counter
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from collections import Counter
from imblearn.over_sampling import RandomOverSampler 

In [2]:
data = Path('./results/DataProcessingExtractFile-RawData.csv')
df = pd.read_csv(data)
df.head(10)

,Diabetes_Status,HighBP,HighChol,CholCheck,BMI_Range,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,Mental_Health_Range,Physical_Health_Range,DiffWalk,Sex,Age,Education,Income
0,1,1.0,1.0,1.0,4,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,4,3,1.0,0.0,9.0,4.0,3.0
1,1,0.0,0.0,0.0,3,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,1,1,0.0,0.0,7.0,6.0,1.0
2,1,1.0,1.0,1.0,3,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,6,6,1.0,0.0,9.0,4.0,8.0
3,1,1.0,0.0,1.0,3,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,1,1,0.0,0.0,11.0,3.0,6.0
4,1,1.0,1.0,1.0,2,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,1,1,0.0,0.0,11.0,5.0,4.0
5,1,1.0,1.0,1.0,3,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,1,1,0.0,1.0,10.0,6.0,8.0
6,1,1.0,0.0,1.0,4,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,3.0,1,3,0.0,0.0,9.0,6.0,7.0
7,1,1.0,1.0,1.0,3,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,3.0,1,1,1.0,0.0,11.0,4.0,4.0
8,2,1.0,1.0,1.0,4,1.0,0.0,1.0,0.0,1.0,...,1.0,0.0,5.0,6,6,1.0,0.0,9.0,5.0,1.0
9,1,0.0,0.0,1.0,2,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,2.0,1,1,0.0,1.0,8.0,4.0,3.0


In [3]:
# K Nearest Neighbors Model

X = df.copy()
X = df.drop(df.columns[[0]], axis=1)
X.head()

,HighBP,HighChol,CholCheck,BMI_Range,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,AnyHealthcare,NoDocbcCost,GenHlth,Mental_Health_Range,Physical_Health_Range,DiffWalk,Sex,Age,Education,Income
0,1.0,1.0,1.0,4,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,5.0,4,3,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,3,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,3.0,1,1,0.0,0.0,7.0,6.0,1.0
2,1.0,1.0,1.0,3,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,5.0,6,6,1.0,0.0,9.0,4.0,8.0
3,1.0,0.0,1.0,3,0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,0.0,2.0,1,1,0.0,0.0,11.0,3.0,6.0
4,1.0,1.0,1.0,2,0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,0.0,2.0,1,1,0.0,0.0,11.0,5.0,4.0


In [4]:
y = df["Diabetes_Status"].ravel()
y[:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 2, 1])

In [5]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y,
                                                    random_state=78,
                                                    stratify=y)
X_train.head()

,HighBP,HighChol,CholCheck,BMI_Range,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,AnyHealthcare,NoDocbcCost,GenHlth,Mental_Health_Range,Physical_Health_Range,DiffWalk,Sex,Age,Education,Income
173928,1.0,0.0,1.0,4,0.0,1.0,0.0,1.0,1.0,1.0,...,1.0,0.0,3.0,1,1,0.0,0.0,11.0,3.0,7.0
239730,0.0,1.0,1.0,3,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,5.0,2,1,1.0,0.0,7.0,3.0,4.0
81199,0.0,1.0,1.0,2,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,2.0,1,1,0.0,0.0,6.0,6.0,8.0
160865,0.0,0.0,1.0,2,0.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,1,1,0.0,1.0,5.0,6.0,8.0
121583,0.0,0.0,1.0,4,0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,0.0,3.0,1,1,1.0,0.0,9.0,6.0,8.0


In [6]:
print('y_train dataset shape {}'.format(Counter(y_train)))

y_train dataset shape Counter({1: 160277, 2: 29983})


In [7]:
# make a balanced dataset before training
ros = RandomOverSampler(random_state=78)
X_res, y_res = ros.fit_resample(X_train, y_train)
print('Resampled dataset shape {}'.format(Counter(y_res)))

Resampled dataset shape Counter({1: 160277, 2: 160277})


In [8]:
# this is very slow - it trains the full model for each test k
# the goal is to find a value in k when the training and testing accuracy are about the same
# result, I didn't find it. going to start with k=9 and see how the rest works out
#K = []
#training = []
#test = []
#scores = {}
#
#for k in range(9, 19):
#	clf = KNeighborsClassifier(n_neighbors = k)
#	clf.fit(X_res, y_res)
#
#	training_score = clf.score(X_train, y_train)
#	test_score = clf.score(X_test, y_test)
#	K.append(k)
#
#	training.append(training_score)
#	test.append(test_score)
#	scores[k] = [training_score, test_score]
#	print(k, ':', scores[k])

In [9]:
# Create a KNN classifier.
knn_model = KNeighborsClassifier(n_neighbors = 75) 

In [10]:
# Fitting the model
knn_model = knn_model.fit(X_res, y_res)

In [11]:
# evaluate the training accuracy first, based on the data before oversampling
pred_train=knn_model.predict(X_train)
cm_train = confusion_matrix(y_train, pred_train)
cmt_df = pd.DataFrame(
    cm_train, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cmt_df

,Predicted 0,Predicted 1
Actual 0,128090,32187
Actual 1,9936,20047


In [12]:
# Calculate the accuracy score on trained set.
acc_score = accuracy_score(y_train, pred_train)
acc_score

0.7786029643645538

In [13]:
# Making predictions using the testing data.
predictions = knn_model.predict(X_test)
predictions

array([1, 1, 1, ..., 1, 1, 2])

In [14]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,42331,11095
Actual 1,3754,6240


In [15]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)
acc_score

0.7658625039419742

In [16]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,42331,11095
Actual 1,3754,6240


Accuracy Score : 0.7658625039419742
Classification Report
              precision    recall  f1-score   support

           1       0.92      0.79      0.85     53426
           2       0.36      0.62      0.46      9994

    accuracy                           0.77     63420
   macro avg       0.64      0.71      0.65     63420
weighted avg       0.83      0.77      0.79     63420



In [17]:
pickle.dump(knn_model, open('knn_model.pkl','wb'))